In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
sns.reset_defaults
sns.set_style(style='darkgrid')
import matplotlib.pyplot as plt
plt.rcParams["patch.force_edgecolor"] = True
plt.rcParams['figure.figsize'] = (20.0, 10.0)
font = {'size'   : 16}
plt.rc('font', **font)
plt.ion()
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import scipy.stats as scs
from sklearn.model_selection import cross_val_score

In [2]:
df = pd.read_csv('data/Train.csv')

/Users/jeremymiller/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.corr()

In [ ]:
df.columns

In [3]:
df_num = df[['SalePrice', 'MachineID', 'ModelID', 'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', ]]

In [4]:
df_num['YearMade'].replace(to_replace = '1000', value = None, inplace=True)

/Users/jeremymiller/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(result._data)


,SalePrice,MachineID,ModelID,auctioneerID,YearMade,MachineHoursCurrentMeter


In [5]:
df_num['MachineHoursCurrentMeter'].replace(to_replace = '0', value = None, inplace=True)

In [ ]:
# df_num = df_num[df_num['MachineHoursCurrentMeter']>0]

In [ ]:
# df_num.info()df_num = df_num[df_num['YearMade']>1900]

In [6]:
df_num.fillna(df_num.median(axis=0), axis=0, inplace=True)

/Users/jeremymiller/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [7]:
df_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 6 columns):
SalePrice                   401125 non-null int64
MachineID                   401125 non-null int64
ModelID                     401125 non-null int64
auctioneerID                401125 non-null float64
YearMade                    401125 non-null int64
MachineHoursCurrentMeter    401125 non-null float64
dtypes: float64(2), int64(4)
memory usage: 18.4 MB


In [ ]:
sns.pairplot(df_num)

In [8]:
X = df_num.drop('SalePrice', axis=1)
y = df_num['SalePrice']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=99)

In [ ]:
# model1 = sm.OLS(y_train, X_train)
# model1_results = model1.fit()
# model1_results.summary()

In [10]:
scaler = StandardScaler()

In [11]:
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [12]:
scaler.transform(X_train)

array([[ 1.00873758e+00, -3.66881387e-01, -3.21645040e-01,
         3.52967543e-01, -7.61791915e-02],
       [-6.60509375e-01,  1.78998059e-02, -2.00866337e-01,
         3.56389860e-01,  8.60688431e-05],
       [-1.97146714e-01, -8.52075209e-01,  1.01080423e-01,
         3.05055101e-01, -7.61791915e-02],
       ...,
       [-2.58339080e-02, -3.51939988e-01, -2.61255688e-01,
         3.32433639e-01, -7.61791915e-02],
       [-2.26042649e+00,  2.75313981e+00, -2.00866337e-01,
         3.56389860e-01,  8.61192495e-01],
       [ 6.54843206e-01, -3.47923483e-01, -3.21645040e-01,
         3.35855956e-01, -7.61791915e-02]])

In [13]:
lm1 = LinearRegression()

In [14]:
lm1.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [15]:
lm1.score(X_train, y_train)

0.07353122469297169

In [16]:
df_num.corr(method='spearman')

,SalePrice,MachineID,ModelID,auctioneerID,YearMade,MachineHoursCurrentMeter
SalePrice,1.000000,-0.183731,-0.263123,-0.060116,0.245355,0.085655
MachineID,-0.183731,1.000000,0.190237,-0.098824,-0.038081,-0.033193
ModelID,-0.263123,0.190237,1.000000,-0.012350,0.095540,0.075665
auctioneerID,-0.060116,-0.098824,-0.012350,1.000000,-0.143583,-0.013127
YearMade,0.245355,-0.038081,0.095540,-0.143583,1.000000,0.326171
MachineHoursCurrentMeter,0.085655,-0.033193,0.075665,-0.013127,0.326171,1.000000


In [ ]:
cross